In [1]:
# Import necessary libraries
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch
import glob
import pickle  # Correct import statement
from tqdm import tqdm
from PIL import Image

# Define function to load images using Pillow library
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

# Load the pretrained ResNet-50 model
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()
# Get preprocessing object for image
preprocess = weights.transforms()

# Define whether to use precomputed embeddings (features) of images
use_precomputed_embeddings = True
emb_filename = 'C:/content/flower_images_embs.pickle'
img_folder = 'C:/content/flowers'

if use_precomputed_embeddings:
    # If embeddings are already computed, load them from the pickle file
    with open(emb_filename, 'rb') as fIn:
        img_names, img_emb_tensors = pickle.load(fIn)
    print("Images:", len(img_names))
else:
    # If embeddings are not computed, extract features from images
    img_names = list(glob.glob(img_folder + '/*/*.jpg'))
    img_emb = []
    # Extract features from images in the dataset (approximately 1 hour)
    for image in tqdm(img_names):
        img_emb.append(
            model(preprocess(pil_loader(image)).unsqueeze(0)).squeeze(0).detach().numpy()
        )
    img_emb_tensors = torch.tensor(img_emb)
    
    # Save the embeddings to a pickle file for later use
    with open(emb_filename, 'wb') as handle:
        pickle.dump([img_names, img_emb_tensors], handle, protocol=pickle.HIGHEST_PROTOCOL)

Images: 4317


In [2]:
!apt install libomp-dev 
!pip install faiss-cpu
import faiss
from sklearn.decomposition import PCA # (PCA - principal component analysis / Анализ главныX компонентов)
# Функция собирает индекс и предварительно уменьшаяет размерность векторов
def build_compressed_index(n_features):
    pca = PCA(n_components=n_features)
    pca.fit(img_emb_tensors)
    compressed_features = pca.transform (img_emb_tensors)
    dataset = np.float32(compressed_features)
    d = dataset.shape[1] # dimension 
    nb = dataset.shape[0] # database size
    xb = dataset
    index_compressed = faiss.IndexFlatL2(d) # build the index
    index_compressed.add(xb) # add vectors to the index
    return [pca, index_compressed]

"apt" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Хэлперы для отображения изображений 
import matplotlib.pyplot as plt 
import matplotlib.image as mping

def main_image(img_path, desc):
    plt.imshow(mping.imread(ing_path))
    plt.xlabel(img_path.split('.')[0] + '_Original Image', fontsize=12) 
    plt.title(desc, fontsize=20)
    plt.show()
               
def similar_images (indices, suptitle):
    plt.figure(figsize=(15,10), facecolor='white')
    plotnumber = 1
    for index in indices [0:4]:
        if plotnumber<-len(indices) :
            ax = plt.subplot(2,2,plotnumber)
            plt.imshow(mping.imread(img_names[index]))
            plt.xlabel(img_names[index], fontsize=12)
            plotnumber+=1
    plt.suptitle(suptitle, fontsize=15) 
    plt.tight_layout()

In [4]:
import numpy as np
# поиск, можно искать по индексу из предварительно извлеченных [изображений или передать новое изображение 
def search (query, factors):
    if (type(query) == str):
        img_path = query
    else:
        img_path = img_names[query]
    one_img_emb = torch.tensor(model(preprocess(read_image(img_path))).unsqueeze(0).squeeze(0).detach().numpy())
    main_image(img_path, 'Query')
    compressor, index_compressed = build_compressed_index(factors)
    D, I = index_compressed.search(np.float32(compressor.transform([one_img_emb.detach().numpy()])),5) 
    similar_images (I[0][1:], "faiss compressed " + str(factors))

img_names[250] это индекс 250 в списке имен файлов, которые были использованы для обучения модели

In [5]:
img_names [250]

'/content/flowers/rose/17062080069_36ac7907d2_n.jpg'

In [6]:
from pprint import pprint
metrics = weights.value.__dict__['meta']['_metrics'] 

print(metrics)

{'ImageNet-1K': {'acc@1': 80.858, 'acc@5': 95.434}}


"search("/content/flowers/daisy/13826249325_f61cb15f86_n.jpg",300)" - зто вызов функции "search" для поиска поxожиx изображений на "/content/flowers/daisy/13826249325_f61cb15f86_n.jpg" с использованием сжатого индекса с числом главных компонент равным 300.